In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import datetime, os
import pandas as pd
import numpy as np

In [ ]:
%load_ext tensorboard

# GPU Setup

In [ ]:
print("Number of GPU's:",len(tf.config.list_physical_devices('GPU')))
!nvidia-smi

In [ ]:
data  = pd.read_csv('/content/drive/MyDrive/Saarthi ai/train_data.csv')
val_data = pd.read_csv('/content/drive/MyDrive/Saarthi ai/valid_data.csv')

In [ ]:
action_labels = list(set(data['action'])) # 6
object_labels = list(set(data['object'])) # 14
location_labels = list(set(data['location'])) # 4
a_map = {action_labels[i] : i for i in range(len(action_labels))}
o_map = {object_labels[i] : i for i in range(len(object_labels))}
l_map = {location_labels[i] : i for i in range(len(location_labels))}

In [ ]:
X = list(data['transcription'])
Y1 = list(data['action'])
Y2 = list(data['object'])
Y3 = list(data['location'])
Y1 = np.array([a_map[x] for x in Y1])
Y2 = np.array([o_map[x] for x in Y2])
Y3 = np.array([l_map[x] for x in Y3])

In [ ]:
val_X = list(val_data['transcription'])
val_Y1 = list(val_data['action'])
val_Y2 = list(val_data['object'])
val_Y3 = list(val_data['location'])
v_Y1 = np.array([a_map[x] for x in val_Y1])
v_Y2 = np.array([o_map[x] for x in val_Y2])
v_Y3 = np.array([l_map[x] for x in val_Y3])

Model

In [ ]:
base_layer_1 = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',trainable = False)

In [ ]:
inputs = tf.keras.layers.Input(shape = [], dtype = tf.string , name = 'Transcription')

base_model_layer = base_layer_1(inputs)
lam_layer = tf.keras.layers.Lambda(lambda x : tf.expand_dims(x, axis =1))(base_model_layer)
x = tf.keras.layers.LSTM(256)(lam_layer)
x = tf.keras.layers.Dense(128, activation= 'relu')(x)


o1 = tf.keras.layers.Dense(6,activation='softmax',name='Action')(x)
o2 = tf.keras.layers.Dense(14,activation='softmax',name='Object')(x)
o3 = tf.keras.layers.Dense(4,activation='softmax',name = 'Location')(x)
out = [o1,o2,o3]

conv_model = tf.keras.Model(inputs, out)
conv_model.summary()

In [ ]:
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
conv_model.compile(loss =tf.keras.losses.sparse_categorical_crossentropy, 
              optimizer = tf.keras.optimizers.Adam(), 
              metrics = 'accuracy')

Training

In [ ]:
conv_model.fit(x=np.array(X),y =(Y1,Y2,Y3),validation_data=(np.array(val_X), (v_Y1,v_Y2,v_Y3)), epochs = 10 , batch_size=64,callbacks=[tensorboard_callback])

In [ ]:
import shutil
shutil.make_archive('logs', 'zip', 'logs')

In [ ]:
%tensorboard --logdir logs

In [ ]:
outs = conv_model.predict(val_X)

In [ ]:
o1 = np.argmax(outs[0],axis=1)
o2 = np.argmax(outs[1],axis=1)
o3 = np.argmax(outs[2],axis=1)

In [ ]:
O1 = [action_labels[x] for x in o1]
O2 = [object_labels[x] for x in o2]
O3 = [location_labels[x] for x in o3]